<a href="https://colab.research.google.com/github/mariya-goltsova/transformers/blob/main/summarization1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# wrap the output in colab cells
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# install transformers with sentencepiece
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00


In [ ]:

# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

Some model statistics

In [ ]:
# max tokens including the special tokens
tokenizer.model_max_length 

1024

In [ ]:
FileContent = open('text.txt').read().strip()
print(FileContent)

In 1948 there were seven thousand people in Grinnell, Iowa, including more than one who didn't dare take a drink in his own house without pulling the shades down first. It was against the law to sell liquor in Grinnell, but it was perfectly legal to drink it at home. So it wasn't that. It wasn't even that someone might look in through the window and disapprove. God knew Grinnell had more than its share of White Ribbon teetotalers, but by 1948 alcohol was hardly the mark of Cain it had once been. No, those timid souls with their fingers through the shade loops inside the white frame houses on Main Street and Park Street were thinking of something else altogether.

They happened to live on land originally owned by the Congregational minister who had founded the town in 1854, Josiah Grinnell. Josiah Grinnell had sold off lots with covenants, in perpetuity, stating that anyone who allowed alcohol to be drunk on his property forfeited ownership. In perpetuity! In perpetuity was forever, and

In [ ]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

151

In [ ]:
len(sentences)

972

In [ ]:
#creating chunks
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk  
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

23

In [ ]:

[len(tokenizer.tokenize(c)) for c in chunks]

[1002,
 1011,
 1018,
 999,
 1021,
 1012,
 1002,
 1002,
 992,
 1002,
 1010,
 977,
 1014,
 1021,
 1012,
 1005,
 992,
 991,
 1019,
 1006,
 1012,
 989,
 809]

In [ ]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 In 1948, Grinnell, Iowa, was one of the last towns in America that people back east would have figured to become the starting point of a bolt into the future that would create the substructure, the electronic grid, of life in the year 2000 and beyond. The town had voted Republican in every presidential election since the first time Abraham Lincoln ran, in 1860.
 Bob Noyce's father, Ralph Sr. was a Congregational minister, his grandfather was a minister. Bob was thrown out of school for a semester because of his clerical lineage. Bob's father was an associate superintendent of the Iowa Conference of Congregations. Bob and his brothers lived just down Park Street and used to rake leaves, mow the lawn, baby-sit, and do other chores for the Gales.
 Bob Noyce was the star diver on the college swimming team and won the Midwest Conference championship in 1947. He sang in choral groups, played the oboe, and was an actor with the college dramatic society. In May 1948 a group of Bob's friends d

вывод: большой объем полученнного текста из-за количества фрагментов(chunks), по каждому из которых составляется сжатие. Предложения добавляются в исходные фрагменты, пока во фрагменте не наберется максимальное количество токенов в предложении (tokenizer.max_len_single_sentence ).

Источник: https://www.youtube.com/watch?v=78KjuKYiF6s